In [ ]:
from spaco_py.SpaCoObject import SPACO
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from momentchi2 import wf, hbe

In [ ]:
sf = np.load(
    "/home/krastega0/SpaCo_py/src/spaco_py/sf_mat.npy", allow_pickle=False
)
neighbor = np.load(
    "/home/krastega0/SpaCo_py/src/spaco_py/A_mat.npy", allow_pickle=False
)

In [ ]:
#Loading the coorddinates 
coords = pd.read_excel('/home/krastega0/SpaCo_py/src/spaco_py/coords_brain.xlsx')

In [ ]:
testObj = SPACO(neighbormatrix=neighbor, sample_features=sf, coords=coords)

In [ ]:
Pspac, Vkt =testObj.spaco_projection()


## Remarks on Fig 1
The distribution of eigenvalues shows the non-random eigenvalues in blue and the random permuted eigenvalues in orange.
The non-random eigenvalues represent the true structure of the data, while the random permuted eigenvalues serve as a null distribution for comparison.
We use the upper 95% confidence interval of the random permuted eigenvalues to determine the significance of the non-random eigenvalues.
The non-random eigenvalues that exceed this threshold are considered significant and indicate the presence of meaningful patterns in the data.
The plot provides a visual representation of the eigenvalue distributions, allowing us to assess the significance of the non-random eigenvalues in relation to the random permuted eigenvalues.
      
      

## Determing a metric to see if this pattern is actually spatially 
Here we see our first spatial pattern but how can we tell if the patterns that we see are actually spatial or they are just artifacts. We look at the length of the projections as the measure of "Smoothness" for a spatial gene 

In [ ]:
graphLaplacian=testObj._cache["spectral_results"][2]
graphLaplacian

In [ ]:
coeefficients = testObj._cache["sigma_eigh"]
coeefficients

In [ ]:
# generating the theoretical null distribution
# first need to create a vector of random chi_squared variables
coeefficients = testObj._cache["sigma_eigh"]
n = len(testObj._cache["sigma_eigh"])
chi2_random = np.random.chisquare(df=1, size=n)

# then we need to scale them by the eigenvalues
theoretical_null = [coeefficients.T @ np.random.chisquare(df=1, size=n) for i in range(10000)]
len(theoretical_null)

In [ ]:
# generate random normal distributed vector
normal_random = np.random.normal(size=Vkt.shape[0])

# generate the test statistic for this random simulated pattern 
rand_test_statistic = []
for i in range(10000):
    rand_projection = np.random.normal(size=Vkt.shape[0]).T @ graphLaplacian @ Vkt
    #rand_projection = Vkt.T @ graphLaplacian @ np.random.normal(size=Vkt.shape[0])
    len_proj = rand_projection.T @ rand_projection
    rand_test_statistic.append(len_proj)
len(rand_test_statistic)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(rand_test_statistic, bins=50, color='blue', alpha=0.6, label='Random Test Statistic')
plt.hist(theoretical_null, bins=50, color='orange', alpha=0.6, label='Theoretical Null')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Comparison of Random Test Statistic and Theoretical Null Distributions')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from scipy.stats import ks_2samp

# Perform the two-sample Kolmogorov-Smirnov (KS) test to determine if the two distributions are significantly different
# theoretical_null and rand_test_statistic are the two samples to compare
# Perform the two-sample KS test
_, p_value = ks_2samp(theoretical_null, rand_test_statistic)

print(f"p-value: {p_value}")

if p_value > 0.05:
    print("The two arrays likely come from the same distribution.")
else:
    print("The two arrays likely come from different distributions.")

## plot spatial heatmap to confirm statistically and visually that the pattern is spatial

In [ ]:
spatially_significant_columns = []
for i in range(testObj.SF.shape[1]):
    
    temp_stat = testObj.SF[:, i].T @ graphLaplacian @ Vkt
    t_spac1 = temp_stat.T @ temp_stat
    if t_spac1 > 1.7:
        spatially_significant_columns.append(i)
len(spatially_significant_columns)

In [ ]:
num_patterns = len(spatially_significant_columns)
fig, axes = plt.subplots(1, num_patterns, figsize=(6 * num_patterns, 6))

if num_patterns == 1:
    axes = [axes]

for ax, idx in zip(axes, spatially_significant_columns):
    values = Pspac[:, idx]
    scatter = ax.scatter(coords['row'], coords['col'], c=values, cmap="viridis", s=20, edgecolor="k")
    ax.set_title(f"SpaCo Pattern {idx}")
    ax.set_xlabel("X Coordinate")
    ax.set_ylabel("Y Coordinate")
    plt.colorbar(scatter, ax=ax, label="Values")

plt.tight_layout()
plt.show()

In [ ]:



p_value, test_stat = testObj.spaco_test(testObj.SF[:, 450])
print(f"p-value: {p_value}, test statistic: {test_stat}")
# Plotting the spatial heatmap for the 450th SpaCo pattern
#testObj.plot_spatial_heatmap(coords, testObj.SF[:, 450], "450th SpaCo pattern", point_size=20)
testObj.coords

In [ ]:
p_value, test_stat = testObj.spaco_test(testObj.SF[:, 2845])
print(f"p-value: {p_value}, test statistic: {test_stat}")
# Plotting the spatial heatmap for the 2845th SpaCo pattern
#plot_spatial_heatmap(coords, testObj.SF[:, 2845], title="2845th SpaCo pattern", point_size=20)
testObj.plot_spatial_heatmap( testObj.SF[:, 2845], "Correctly Oriented plot SpaCo pattern (Ttr)", cmap="viridis", point_size=20)

In [ ]:
p_value, test_stat = testObj.spaco_test(testObj.SF[:, 450])
print(f"p-value: {p_value}, test statistic: {test_stat}")
# Plotting the spatial heatmap for the 450th SpaCo pattern
#plot_spatial_heatmap(coords, testObj.SF[:, 450], title="450th SpaCo pattern", point_size=20)
testObj.plot_spatial_heatmap( testObj.SF[:, 450], "Non-Spatial Pattern", cmap="viridis", point_size=20)